<div style="direction:rtl; text-align:right">
أهلاً بكم في المشروع السادس من علم البيانات. في هذا المشروع، سنقوم معاً بالنظر إلى بيانات فريدة من نوعها وهي مجموعة من النصوص المصنفة إلى مجالات مختلفة. سنقوم بنمذجة هذه البيانات لتوقع تصنيف النصوص بناء على الكلمات الواردة. تستطيع قراءة المزيد عن البيانات من خلال موقع سايكت-ليرن:

- https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

<div style="direction:rtl; text-align:right">
ملاحظة: عليك تعديل الخانات التي يوجد فيها الكود أدناه قبل تسليم المشروع. جميع الخانات الأخرى يجب ان تبقى كما هي بدون أي 
تعديل.

```
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
```

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import string
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mais\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

<div style="direction:rtl; text-align:right">

# قراءة البيانات

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
newsgroups_data = fetch_20newsgroups(subset='train')

In [4]:
X = newsgroups_data['data']
y = newsgroups_data['target']

<div style="direction:rtl; text-align:right">
لاحظ أن X هنا عبارة عن قائمة بالنصوص، وليست جدول كما هو الحال مع الدروس الأخرى. أما y فهي كذلك قائمة بالتصنيفات المختلفة. لمعرفة معنى التصنيفات نستطيع طباعة target_names:

In [5]:
for i, k in enumerate(newsgroups_data['target_names']):
    print(i, '\t->\t', k)

0 	->	 alt.atheism
1 	->	 comp.graphics
2 	->	 comp.os.ms-windows.misc
3 	->	 comp.sys.ibm.pc.hardware
4 	->	 comp.sys.mac.hardware
5 	->	 comp.windows.x
6 	->	 misc.forsale
7 	->	 rec.autos
8 	->	 rec.motorcycles
9 	->	 rec.sport.baseball
10 	->	 rec.sport.hockey
11 	->	 sci.crypt
12 	->	 sci.electronics
13 	->	 sci.med
14 	->	 sci.space
15 	->	 soc.religion.christian
16 	->	 talk.politics.guns
17 	->	 talk.politics.mideast
18 	->	 talk.politics.misc
19 	->	 talk.religion.misc


<div style="direction:rtl; text-align:right">
وهنا سنقوم بطباعة أول قيمة من X

In [6]:
print(X[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [14]:
s=X[0]
s = re.sub(r"[^a-z]"," ",s.lower())
print(s)

#punctuations
#punct=string.punctuation
#print(punct)

from  lerxst wam umd edu  where s my thing  subject  what car is this   nntp posting host  rac  wam umd edu organization  university of maryland  college park lines       i was wondering if anyone out there could enlighten me on this car i saw the other day  it was a   door sports car  looked to be from the late   s  early   s  it was called a bricklin  the doors were really small  in addition  the front bumper was separate from the rest of the body  this is  all i know  if anyone can tellme a model name  engine specs  years of production  where this car is made  history  or whatever info you have on this funky looking car  please e mail   thanks    il         brought to you by your neighborhood lerxst          


<div style="direction:rtl; text-align:right">
لنقم أولاً بعرض كم عدد النصوص لكل صنف ؟ قد تحتاج لاستخدام باندا للقيام بالمهمة!

In [ ]:
numbers_to_categories={0: 'alt.atheism', 1: 'comp.graphics', 2: 'comp.os.ms-windows.misc', 3: 'comp.sys.ibm.pc.hardware', 4: 'comp.sys.mac.hardware', 5: 'comp.windows.x', 6: 'misc.forsale', 7: 'rec.autos', 8: 'rec.motorcycles', 9: 'rec.sport.baseball', 10: 'rec.sport.hockey', 11: 'sci.crypt', 12: 'sci.electronics', 13: 'sci.med', 14: 'sci.space', 15: 'soc.religion.christian', 16: 'talk.politics.guns', 17: 'talk.politics.mideast', 18: 'talk.politics.misc', 19: 'talk.religion.misc'}

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################



print (len(y))
df_c=pd.DataFrame(columns=['category'])
df_c['category']= pd.Series(y)
df_c['category'].value_counts()
#df_c['category'].replace(numbers_to_categories, inplace=True)

<div style="direction:rtl; text-align:right">
نرى أن لدينا توازن في توزيع النصوص بين الأصناف المختلفة

<div style="direction:rtl; text-align:right">

# تجهيز البيانات

<div style="direction:rtl; text-align:right">
ماذا نعمل عادة مع البيانات النصية؟ في هذه الحالة، سيكون أمامنا الكثير لنعمله حتى نقوم بتنظيف هذه البيانات وتهيئتها للتحليل. لذلك سنقوم بكتابة بعض الأوامر التي تساعدنا على ذلك.

<div style="direction:rtl; text-align:right">
المهمة الأولى: تنظيف النص

In [67]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################


# HINT: figure out how to: delete special charachters, delete numbers, delete email addresses
#          if you dont know, use google!


def clean_text(text):

    text = re.sub(r"[^a-z]"," ",text.lower())
    text = re.sub(' +'," ",text )



    return text

<div style="direction:rtl; text-align:right">
حسنا، سنقوم الآن برؤية أثر هذه التعديلات على العنصر الأول في قائمة النصوص

In [69]:
print(clean_text(X[0]))

from lerxst wam umd edu where s my thing subject what car is this nntp posting host rac wam umd edu organization university of maryland college park lines i was wondering if anyone out there could enlighten me on this car i saw the other day it was a door sports car looked to be from the late s early s it was called a bricklin the doors were really small in addition the front bumper was separate from the rest of the body this is all i know if anyone can tellme a model name engine specs years of production where this car is made history or whatever info you have on this funky looking car please e mail thanks il brought to you by your neighborhood lerxst 


<div style="direction:rtl; text-align:right">
سنقوم الآن بتنفيذ ذات الأمر على باقي النصوص في X:

In [26]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

for i in range(len(X)):
    try:
        X[i]=clean_text(X[i])
        if X[i]=="":
            continue
    except UnicodeDecodeError:
        continue
        

In [27]:
print(X[1])

from guykuo carson u washington edu guy kuo subject si clock poll final call summary final call for si clock reports keywords si acceleration clock upgrade article i d shelley qvfo innc s organization university of washington lines nntp posting host carson u washington edu a fair number of brave souls who upgraded their si clock oscillator have shared their experiences for this poll please send a brief message detailing your experiences with the procedure top speed attained cpu rated speed add on cards and adapters heat sinks hour of usage per day floppy disk functionality with and m floppies are especially requested i will be summarizing in the next two days so please add to the network knowledge base if you have done the clock upgrade and haven t answered this poll thanks guy kuo guykuo u washington edu 


<div style="direction:rtl; text-align:right">
الآن سنقوم باستخدام Td-IDF لتحويل الكلمات إلى مصفوفة ضخمة حتى نستخدمها في النمذجة. في الخانة التالية، ستقوم باستخدام TfidfVectorizer لتحويل النص.

In [80]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
from sklearn.feature_extraction.text import TfidfVectorizer

#stop = stopwords.words('english')

vec = TfidfVectorizer()

trn_term_doc = vec.fit_transform(X)

bag_of_words=pd.DataFrame(trn_term_doc.toarray(),columns=vec.get_feature_names())
np.round(bag_of_words,2)



MemoryError: 

In [ ]:
trn_term_doc

<div style="direction:rtl; text-align:right">

# نمذجة البيانات

<div style="direction:rtl; text-align:right">
سنقوم الآن بنمذجة البيانات. لنبدأ بالانحدار اللوجستي. في الخانة أدناه، قم باستيراد كلاً من مكتبة الانحدار اللوجستي، وأمر cross_val_score ثم استخدمهم في تدريب النموذج والتحقق من صحته.

In [77]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

# HINT: You should use `train_data` as your input
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
model = LogisticRegression()
scores = cross_val_score(model,X,y,cv=5)
print(scores)

[nan nan nan nan nan]


C:\Users\mais\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'from hall vice hal f lillywhite lp a yapg subject re help organization tektronix inc beaverton or lines in article apr geneva rutgers edu lmvec westminster ac uk william hargreaves writes i m a commited christian that is battling with a problem i know that romans talks about how we are saved by our faith not our deeds yet hebrews and james say that faith without deeds is useless saying you fools do you still think that just believing is enough actually i don t think there is any conflict if we really understand what these passages say first what is faith if you study the meaning of the greek and hebrew words so translated i think you will come to the conclusion that the word means a lot more than mere belief faith means both t

In [81]:
avg_score = scores.mean()

In [82]:
print('Average accuracy is: ', avg_score)

Average accuracy is:  nan


<div style="direction:rtl; text-align:right">
لا تحتاج في هذا المشروع إلى تسليم أي ملف إلى كاقل. ولكن تأكد من صحة الإجابات ثم قم بتسليم المشروع في الموقع.